# Register File Generator in Magma

This notebook walks through the design of a register file generator in magma.

First, we'll define the interface to our register file generator as a set of
parameters:
* `reg_list` - a list of objects describing the internal registers of the
  register file
* `data_width` - the width of each register
* `apb_slave_id` - the slave ID of the register file for interfacing with an
  APB bus

Now, we'll define the a `Register` object in standard Python that will be used
to describe the internal registers

In [1]:
class Register:
    def __init__(self, name: str, init: int=0, has_ce: bool=False):
        """
        name:   string corresponding to the name of the register
        init:   integer corresponding to the initial value of the register
        has_ce: boolean selecting whether the register has a clock enable
                signal
        """
        self.name = name
        self.init = init
        self.has_ce = has_ce

Next, we'll define a helper function for creating the magma interface for the
generated register file circuit. Magma interfaces are described as a list of
the form `["<port_name>", <port_type>, ...]`. To do this, we'll first need to
define some magma types for constructing an APB interface.

In [2]:
import math
import magma as m

def APBBase(addr_width: int, data_width: int):
    """
    Constructs a dictionary mapping port names to magma types
    
    Used to construct the master and slave variants of the APB inteface
    
    Parametrized by width of the address and data bus
    """
    return {
        "PCLK"   : m.Out(m.Clock),
        "PRESETn": m.Out(m.Reset),
        "PADDR"  : m.Out(m.Bits[addr_width]),
        "PPROT"  : m.Out(m.Bit),
        "PENABLE": m.Out(m.Bit),
        "PWRITE" : m.Out(m.Bit),
        "PWDATA" : m.Out(m.Bits[data_width]),
        # One write strobe bit for each byte of the data bus
        "PSTRB"  : m.Out(m.Bits[math.ceil(data_width / 8)]),
        "PREADY" : m.In(m.Bit),
        "PRDATA" : m.In(m.Bits[data_width]),
        "PSLVERR": m.In(m.Bit),
    }

def APBMaster(addr_width: int, data_width: int, num_sel: int=1):
    """
    Constructs the master variant of the APB interface using APBBase
    
    Parametrized by the width of the address and data bus as well as the number
    of slaves (`num_sel`)
    """
    if not data_width <= 32:
        raise ValueError("AMBA 3 APB specifies that the data bus " \
                         "cannot be wider than 32 bits")
    
    # Construct dictionary with a PSELx port for each slave
    PSELs = {}
    for i in range(num_sel):
        PSELs[f"PSEL{i}"] = m.Out(m.Bit)
        
    # Concatenate the APBBase dictionary with the PSEL dictionary to 
    # generate the full interface
    return m.Tuple(
        **APBBase(addr_width, data_width),
        **PSELs,
    )


from typing import List, Union

def APBSlave(addr_width: int, data_width: int, 
             slave_id_or_ids: Union[int, List[int]]):
    """
    Constructs the slave variant of the APB interface using APBBase
    
    Parametrized by the with of the address and data bus as well as
    the slave id or a list of slave ids (to support lifting the
    APBSlave interface for a module that contains multiple slave 
    instances)
    
    `slave_id_or_ids` is either an id (e.g. 2) or a list of ids ([0, 1, 2])
    """
    # If the `slave_id_or_ids` parameter is a single integer, we convert it to
    # a list of a single integer so the rest of the code can assume that it is
    # a list of integers, otherwise, we check that it is a list of integers
    if isinstance(slave_id_or_ids, int):
        slave_id_or_ids = [slave_id_or_ids]
    elif not isinstance(slave_id_or_ids, list) and \
         all(isinstance(x, int) for x in slave_or_slave_ids):
        raise ValueError(f"Received incorrect parameter for "
                         f"`slave_or_slave_ids`: {slave_or_slave_ids}")
    
    # APBBase is defined in terms of the master, so we define PSEL as an output
    # since the entire type will be flipped
    PSEL = {f"PSEL{slave_id}": m.Out(m.Bit) for slave_id in slave_id_or_ids}
    
    # Note the use of `flip()` to return the inverse of the type created by
    # APBBase
    return m.Tuple(**APBBase(addr_width, data_width), **PSEL).flip()

Let's look at some simple examples of using our type constructors

In [3]:
m.util.pretty_print_type(APBMaster(addr_width=16, data_width=32, num_sel=2))

Tuple(
    PCLK = Out(Clock),
    PRESETn = Out(Reset),
    PADDR = Out(Bits[16]),
    PPROT = Out(Bit),
    PENABLE = Out(Bit),
    PWRITE = Out(Bit),
    PWDATA = Out(Bits[32]),
    PSTRB = Out(Bits[4]),
    PREADY = In(Bit),
    PRDATA = In(Bits[32]),
    PSLVERR = In(Bit),
    PSEL0 = Out(Bit),
    PSEL1 = Out(Bit)
)


In [4]:
m.util.pretty_print_type(APBSlave(addr_width=16, data_width=32,
                                  slave_id_or_ids=[0, 1]))

Tuple(
    PCLK = In(Clock),
    PRESETn = In(Reset),
    PADDR = In(Bits[16]),
    PPROT = In(Bit),
    PENABLE = In(Bit),
    PWRITE = In(Bit),
    PWDATA = In(Bits[32]),
    PSTRB = In(Bits[4]),
    PREADY = Out(Bit),
    PRDATA = Out(Bits[32]),
    PSLVERR = Out(Bit),
    PSEL0 = In(Bit),
    PSEL1 = In(Bit)
)


With our APB type constructors defined, we can now create a constructor for the
register file interface

In [5]:
def make_reg_file_interface(reg_list: List[Register], data_width: int,
                            apb_slave_id: int):
    # magma provides various helper functions in m.bitutils, 
    # here we use clog2 to derive the number of bits required 
    # to store the address space described by number of Registers 
    # in `reg_list`
    addr_width = m.bitutils.clog2(len(reg_list))
    
    Data = m.Bits[data_width]
    
    IO = ["apb", APBSlave(addr_width, data_width, apb_slave_id)]
    for reg in reg_list:
        IO += [f"{reg.name}_d", m.In(Data)]
        if reg.has_ce:
            IO += [f"{reg.name}_en", m.In(m.Enable)]
        IO += [f"{reg.name}_q", m.Out(Data)]
    return IO

Let's try it out

In [6]:
interface = make_reg_file_interface(reg_list=[Register("reg0", 1, True),
                                    Register("reg1", 24)], data_width=32,
                                    apb_slave_id=1)

# Print the interface two elements at a time
for i in range(0, len(interface), 2):
    print(f"port_name = \"{interface[i]}\"")
    print(f"port_type = ", end="")
    m.util.pretty_print_type(interface[i + 1])
    print()

port_name = "apb"
port_type = Tuple(
    PCLK = In(Clock),
    PRESETn = In(Reset),
    PADDR = In(Bits[1]),
    PPROT = In(Bit),
    PENABLE = In(Bit),
    PWRITE = In(Bit),
    PWDATA = In(Bits[32]),
    PSTRB = In(Bits[4]),
    PREADY = Out(Bit),
    PRDATA = Out(Bits[32]),
    PSLVERR = Out(Bit),
    PSEL1 = In(Bit)
)

port_name = "reg0_d"
port_type = In(Bits[32])

port_name = "reg0_en"
port_type = In(Enable)

port_name = "reg0_q"
port_type = Out(Bits[32])

port_name = "reg1_d"
port_type = In(Bits[32])

port_name = "reg1_q"
port_type = Out(Bits[32])



Notice that we have a port named `apb` with an instance of the `APBSlave` type,
followed by interface ports for each register (`reg0`, `reg`) where
`<reg_name>_d` is the input port, `<reg_name>_q` is the output port, and
`<reg_name>_en` is the enable port (if it has one).

Now we can define the generator for a Register file circuit.

In [7]:
import mantle

def DefineRegFile(reg_list, data_width, apb_slave_id=0):
    """
    reg_list : list of Register instances
    """
    class RegFile(m.Circuit):
        name = "RegFile_" + "_".join(reg.name for reg in reg_list)
        IO = make_reg_file_interface(reg_list, data_width, apb_slave_id)

        @classmethod
        def definition(io):
            # Get the concrete PSEL signal based on the `apb_slave_id`
            # parameter
            PSEL = getattr(io.apb, f"PSEL{apb_slave_id}")
            
            # Create a list of Register instances (parametrized by the members
            # of `reg_list`)
            registers = [
                mantle.Register(data_width, init=reg.init, has_ce=True,
                                has_reset=True, name=reg.name)
                for reg in reg_list
            ]
            
            is_write = io.apb.PENABLE & io.apb.PWRITE & PSEL
            
            ready = None
            for i, reg in enumerate(registers):
                # Register input is from `<reg_name>_d` port by default
                # and PWDATA when handling an APB write
                reg.I <= mantle.mux([getattr(io, reg.name + "_d"),
                                     io.apb.PWDATA], is_write)
                
                # Wire up register output to `<reg_name>_q` interface port
                getattr(io, reg.name + "_q") <= reg.O
                
                # Wire the clock signals
                reg.CLK <= io.apb.PCLK
                reg.RESET <= ~m.bit(io.apb.PRESETn)
                
                # Clock enable is based on write signal and PADDR value
                # For now, a register's address is defined by its index in
                # `reg_list`
                ce = is_write & (io.apb.PADDR == i)
                if reg_list[i].has_ce:
                    # If has a clock enable, `or` the enable signal with the IO
                    # input
                    reg.CE <= ce | m.bit(getattr(io, reg.name + "_en"))
                else:
                    reg.CE <= ce

                # Set ready high if a register is being written to
                if ready is not None:
                    ready |= ce
                else:
                    ready = ce
                    
            is_read = io.apb.PENABLE & ~io.apb.PWRITE & PSEL
            
            # PREADY is high if a write or read is being performed
            io.apb.PREADY <= ready | is_read
            
            # Select PRDATA based on PADDR
            io.apb.PRDATA <= mantle.mux(
                [reg.O for reg in registers], io.apb.PADDR)
            
            # Stub out the rest of the signals for now, CoreIR does not allow
            # unconnected signals, so we wire them up to the CoreIR `Term`
            # module which is a stub module that takes a single input and no
            # output (effectively "casting" the unwired port so the compiler
            # does not complain)
            io.apb.PSLVERR <= 0
            mantle.CorebitTerm().I <= io.apb.PPROT
            mantle.Term(len(io.apb.PSTRB)).I <= io.apb.PSTRB

    return RegFile

Let's try it out

In [8]:
RegFile = DefineRegFile([Register("reg0", 1, True), Register("reg1", 24)],
                         data_width=32, apb_slave_id=1)

# print the magma internal representation of the circuit
print(repr(RegFile))
print("Hello")

RegFile_reg0_reg1 = DefineCircuit("RegFile_reg0_reg1", "apb", Tuple(PCLK=In(Clock),PRESETn=In(Reset),PADDR=In(Bits[1]),PPROT=In(Bit),PENABLE=In(Bit),PWRITE=In(Bit),PWDATA=In(Bits[32]),PSTRB=In(Bits[4]),PREADY=Out(Bit),PRDATA=Out(Bits[32]),PSLVERR=Out(Bit),PSEL1=In(Bit)), "reg0_d", In(Bits[32]), "reg0_en", In(Enable), "reg0_q", Out(Bits[32]), "reg1_d", In(Bits[32]), "reg1_q", Out(Bits[32]))
Mux2xOutBits32_inst0 = Mux2xOutBits32()
Mux2xOutBits32_inst1 = Mux2xOutBits32()
Mux2xOutBits32_inst2 = Mux2xOutBits32()
and_inst0 = and()
and_inst1 = and()
and_inst2 = and()
and_inst3 = and()
and_inst4 = and()
and_inst5 = and()
corebit_term_inst0 = corebit_term()
coreir_eq_1_inst0 = coreir_eq_1()
coreir_eq_1_inst1 = coreir_eq_1()
not_inst0 = not()
not_inst1 = not()
not_inst2 = not()
or_inst0 = or()
or_inst1 = or()
or_inst2 = or()
reg0 = Register_has_ce_True_has_reset_True_has_async_reset_False_type_Bits_n_32(name="reg0")
reg1 = Register_has_ce_True_has_reset_True_has_async_reset_False_type_Bits_n_32(

We can also inspect the generated Verilog

In [9]:
m.compile("RegFile", RegFile, output="coreir-verilog")
with open("RegFile.v", "r") as verilog:
    print(verilog.read())

module coreir_term #(parameter width=1) (
  input [width-1:0] in
);


endmodule  // coreir_term

module coreir_reg #(parameter clk_posedge=1, parameter init=1, parameter width=1) (
  input clk,
  input [width-1:0] in,
  output [width-1:0] out
);
  reg [width-1:0] outReg=init;
  wire real_clk;
  assign real_clk = clk_posedge ? clk : ~clk;
  always @(posedge real_clk) begin
    outReg <= in;
  end
  assign out = outReg;

endmodule  // coreir_reg

module coreir_mux #(parameter width=1) (
  input [width-1:0] in0,
  input [width-1:0] in1,
  input sel,
  output [width-1:0] out
);
  assign out = sel ? in1 : in0;

endmodule  // coreir_mux

module coreir_eq #(parameter width=1) (
  input [width-1:0] in0,
  input [width-1:0] in1,
  output out
);
  assign out = in0 == in1;

endmodule  // coreir_eq

module coreir_const #(parameter value=1, parameter width=1) (
  output [width-1:0] out
);
  assign out = value;

endmodule  // coreir_const

module corebit_term (
  input in
);


endmodule  // corebit_

In the following notebooks, we'll define a Python model for an APBMaster and
use it to functionally verify our register file generator with magma's
verification package `fault`. Then, we'll show an example of using the register
file in a larger design.